# Alternative Framing

In the [main notebook](../robust-genetics.ipynb) it's mentioned that the mathematics is more complicated when the robust optimization problem is framed as a risk minimization problem, rather than return maximization as done there. This notebook includes the calculations of the alternative framing to illustrate that point.

In [1]:
import numpy as np                  # defines matrix structures
from qpsolvers import solve_qp      # used for quadratic optimization
import gurobipy as gp               # Gurobi optimization interface (1)
from gurobipy import GRB            # Gurobi optimization interface (2)

# want to round rather than truncate when printing
np.set_printoptions(threshold=np.inf)

# only show numpy output to five decimal places
np.set_printoptions(formatter={'float_kind':"{:.5f}".format})

## Standard Problem

In the context of genetic selection, we want to maximize selection of genetic merit (a measure of desirable traits) while minimizing risks due to inbreeding. This can be formed mathematically as a problem about minimizing risk for varying levels of return. We say
$$
    \min_w \left(\frac{1}{2}w^{T}\Sigma w - \lambda w^{T}\mu\right)\ \text{ subject to }\ Mw = \begin{bmatrix} 0.5 \\ 0.5\end{bmatrix},\ l\leq w\leq u,
$$
where $w$ is the vector of proportional contributions, $\Sigma$ is a matrix encoding risk, $\mu$ is a vector encoding returns, $l$ encodes lower bounds on contributions, $u$ encodes upper bounds on contributions, and $M$ is matrix which encodes whether each candidate is a sire or dam as described previously.

In this representation of the problem, $\lambda$ is a control variable which balances how we trade of between risk and return. Each value of $\lambda$ will give a different solution on the critical frontier of the problem.

### Toy Example ($n = 3$)

Lets see how this works in an example. We will start by looking how this problem might be solving using Python's [qpsolvers](https://qpsolvers.github.io/qpsolvers/index.html) library. Consider the problem where
$$
    \mu = \begin{bmatrix} 1 \\ 5 \\ 2 \end{bmatrix},\quad
    \Sigma = \begin{bmatrix} 1 & 0 & 0 \\ 0 & 5 & 0 \\ 0 & 0 & 3 \end{bmatrix}, \quad
    \mathcal{S} = \lbrace 1 \rbrace, \quad
    \mathcal{D} = \lbrace 2, 3 \rbrace, \quad
    l = {\bf 0}, \quad
    u = {\bf 1}.
$$
We define these variables in Python using the following code.

In [2]:
# KEY PROBLEM VARIABLES
problem_size = 3
expected_breeding_values = np.array([
    1.0,
    5.0,
    2.0
])
relationship_matrix = np.array([
    [1, 0, 0],
    [0, 5, 0],
    [0, 0, 3]
])
sire_indices = [0]
dam_indices  = [1,2]
lower_bound = np.full((problem_size, 1), 0.0)
upper_bound = np.full((problem_size, 1), 1.0)

We have the additional variables which need setting up so that the problem works in `qpsolvers`. 

In [3]:
# OPTIMIZATION SETUP VARIABLES
lam = 0.5
# define the M so that column i is [1;0] if i is a sire and [0;1] otherwise 
M = np.zeros((2, problem_size))
M[0, sire_indices] = 1
M[1, dam_indices] = 1
# define the right hand side of the constraint Mx = m
m = np.array([[0.5], [0.5]])

Finally, we solve the problem using the modules' `solve_qp` function. This utilises Gurobi via an API, a fact which will be important once we start to consider larger problem sizes. 

In [4]:
# SOLVE THE PROBLEM
def solveGS(lam):
    return solve_qp(
        P = relationship_matrix,
        q = -lam*expected_breeding_values,
        G = None,
        h = None,
        A = M,
        b = m,
        lb = lower_bound,
        ub = upper_bound,
        solver = "gurobi"
    )

print(f"QP solution: w = {solveGS(lam)}")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-26
QP solution: w = [0.50000 0.37500 0.12500]


Excellent, we have code which is telling us that our optimal contributions for $\lambda = 0.5$ are $w_1 = 0.5$, $w_2 = 0.375$, and $w_3 = 0.125$. We could vary our $\lambda$ value too and find other points on the frontier.

In [5]:
print(f"lambda = 0.0: w = {solveGS(0.0)}")
print(f"lambda = 0.2: w = {solveGS(0.2)}")
print(f"lambda = 0.4: w = {solveGS(0.4)}")
print(f"lambda = 0.6: w = {solveGS(0.6)}")
print(f"lambda = 0.8: w = {solveGS(0.8)}")
print(f"lambda = 1.0: w = {solveGS(1.0)}")

lambda = 0.0: w = [0.50000 0.18750 0.31250]
lambda = 0.2: w = [0.50000 0.26250 0.23750]
lambda = 0.4: w = [0.50000 0.33750 0.16250]
lambda = 0.6: w = [0.50000 0.41250 0.08750]
lambda = 0.8: w = [0.50000 0.48750 0.01250]
lambda = 1.0: w = [0.50000 0.50000 0.00000]


## Robust Optimization

As discussed, rather than treating $\mu$ as known we must model it using a univariate normal distribution, $\mu\sim N(\bar{\mu}, \Omega)$. We adjust the objective function to model the inherent uncertainty in the problem, and in the risk-minimization framing the problem the bilevel optimization problem is
$$
    \min_w \left(\frac{1}{2}w^{T}\Sigma w - \lambda\max_{\mu\in U_{\mu}} \left(w^{T}\mu \right)\right)\ \text{ subject to }\ Mw = \begin{bmatrix} 0.5 \\ 0.5\end{bmatrix},\ l\leq w\leq u.
$$


### Quadratic Uncertainty Sets

We define $U_\mu$ as a quadratic uncertainty set,
$$
    U_{\mu} := \left\lbrace \mu :\ {(\mu-\bar{\mu})}^{T}\Omega^{-1}(\mu-\bar{\mu}) \leq \kappa^2 \right\rbrace.
$$
This means that our lower level problem $\max_{\mu\in U_{\mu}} w^{T}\mu$ becomes
$$
    \max_{\mu} w^{T}\mu\quad \text{subject to}\quad {(\mu-\bar{\mu})}^{T}\Omega^{-1}(\mu-\bar{\mu}) \leq \kappa^2,
$$
or, in standard form,
$$
    \min_{\mu} \left(-w^{T}\mu\right) \quad\text{subject to}\quad \kappa^2 - {(\mu-\bar{\mu})}^{T}\Omega^{-1}(\mu-\bar{\mu}) \geq 0.
$$
This is convex, since $\Omega$ is a positive definite matrix and $-w^{T}\mu$ is an affine function. If we define a Lagrangian multiplier $\rho\in\mathbb{R}$, the KKT conditions of this problem are:
\begin{align}
    \nabla_{\mu}L(\mu, \rho) = 0 \quad&\Rightarrow\quad \nabla_{\mu}\left( -w^{T}\mu - \rho\big( \kappa^2 - {(\mu-\bar{\mu})}^{T}\Omega^{-1}(\mu-\bar{\mu}) \big) \right) = 0, \\
    c(\mu) \geq 0                \quad&\Rightarrow\quad \kappa^2 - {(\mu-\bar{\mu})}^{T}\Omega^{-1}(\mu-\bar{\mu}) \geq 0, \\
    \rho \geq 0                  \quad&\Rightarrow\quad \rho\geq0, \\
    \rho c(\mu) = 0              \quad&\Rightarrow\quad \rho\left(\kappa^2 - {(\mu-\bar{\mu})}^{T}\Omega^{-1}(\mu-\bar{\mu}) \right) = 0.
\end{align}
From (1) we have that
$$
    -w + \rho 2\Omega^{-1}(\mu-\bar{\mu}) = 0 \quad\Rightarrow\quad \mu - \bar{\mu} = \frac{1}{2\rho}\Omega w \qquad (5)
$$
which when substituted into (4) gives
\begin{align*}
    &\phantom{\Rightarrow}\quad \rho\left( \kappa^2 - {(\mu-\bar{\mu})}^{T}\Omega^{-1}(\mu-\bar{\mu}) \right) = 0 \\
    &\Rightarrow\quad \rho\left( \kappa^2 - \big(\frac{1}{2\rho}\Omega w\big)^{T}\Omega^{-1}\big(\frac{1}{2\rho}\Omega w\big) \right) = 0 \\
    &\Rightarrow\quad \rho\kappa^2 - \frac{1}{4\rho}w^{T}\Omega^{T}\Omega^{-1}\Omega w = 0 \\
    &\Rightarrow\quad \rho^2\kappa^2 = \frac{1}{4}w^{T}\Omega w\quad \text{(since $\Omega$ symmetric)} \\
    &\Rightarrow\quad \rho\kappa = \frac{1}{2}\sqrt{w^{T}\Omega w}\quad \text{(since $\rho,\kappa\geq0$)} \\
    &\Rightarrow\quad \frac{1}{2\rho} = \frac{\kappa}{\sqrt{w^{T}\Omega w}}.
\end{align*}
Substituting this back into (5), we find that
$$
    \mu - \bar{\mu} = \frac{\kappa}{\sqrt{w^{T}\Omega w}}\Omega w \quad\Rightarrow\quad \mu = \bar{\mu} + \frac{\kappa}{\sqrt{w^{T}\Omega w}}\Omega w
$$
as the solution for the inner problem. Substituting this back into the outer problem gives
$$
    \min_w \left(\frac{1}{2}w^{T}\Sigma w - \lambda w^{T}\left( \bar{\mu} + \frac{\kappa}{\sqrt{w^{T}\Omega w}}\Omega w \right)\right)\ \text{ subject to }\ Mw = \begin{bmatrix} 0.5 \\ 0.5\end{bmatrix},\ l\leq w\leq u.
$$
which after simplifying down and rationalizing the denominator becomes
$$
    \min_w \left(\frac{1}{2}w^{T}\Sigma w - \lambda w^{T}\bar{\mu} - \lambda\kappa\sqrt{w^{T}\Omega w}\right)\ \text{ subject to }\ Mw = \begin{bmatrix} 0.5 \\ 0.5\end{bmatrix},\ l\leq w\leq u
$$
where $\kappa\in\mathbb{R}$ is our robust optimization parameter. Since our objective has gained an additional square root term, this is obviously no longer a quadratic problem and `qpsolvers` is no longer a viable tool. We will instead now need to work with the Gurobi API directly.

### Using Gurobi

To illustrate how the setup changes when uncertainty is added, we will first look at how Gurobi handles the standard problem. The following code returns the same solution as 

In [6]:
# create a model for standard genetic selection
model = gp.Model("standardGS")

# define variable of interest as a continuous 
w = model.addMVar(shape=problem_size, lb=0.0, vtype=GRB.CONTINUOUS, name="w")

# set the objective function
model.setObjective(
    0.5*w.transpose()@(relationship_matrix@w) - lam*w.transpose()@expected_breeding_values,
GRB.MINIMIZE)

# add sub-to-half constraints
model.addConstr(M @ w == m, name="sum-to-half")
# add weight-bound constraints
model.addConstr(w >= lower_bound, name="lower bound")
model.addConstr(w <= upper_bound, name="upper bound")

# solve the problem with Gurobi
model.optimize()
print(f"w = {w.X}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i5-8350U CPU @ 1.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 22 rows, 3 columns and 24 nonzeros
Model fingerprint: 0xd96d3c43
Model has 3 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 2e+00]
  QObjective range [1e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 1e+00]
Presolve removed 21 rows and 1 columns
Presolve time: 0.01s
Presolved: 1 rows, 2 columns, 2 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     

Unfortunately Gurobi cannot handle our problem with its objective function in the form
$$
    \min_w \left(\frac{1}{2}w^{T}\Sigma w - \lambda w^{T}\mu - \lambda\kappa\sqrt{w^{T}\Omega w}\right)\ \text{ subject to }\ Mw = \begin{bmatrix} 0.5 \\ 0.5\end{bmatrix},\ l\leq w\leq u,
$$
so further adjustments are needed first. If we define a real auxillary variable $z\geq0$ such that $z\leq\sqrt{w^{T}\Omega w}$, then our problem becomes
$$
    \min_w \left(\frac{1}{2}w^{T}\Sigma w - \lambda w^{T}\mu - \lambda\kappa z\right)\ \text{ s.t. }\ z\leq\sqrt{w^{T}\Omega w},\ Mw = \begin{bmatrix} 0.5 \\ 0.5\end{bmatrix},\ l\leq w\leq u.
$$
Since $\kappa,\lambda\geq0$, and we're minimizing an objective containing "$-\lambda\kappa z$", this term of the objective will be smallest when $z>0$ is biggest. This happens precisely when it attains its upper bound from the constraint $z\leq\sqrt{w^{T}\Omega w}$, hence our relaxation is justified since $z$ will push upwards.

However, Gurobi _still_ can't handle this due to the presence of the square root, so we further make use of both $z$ and $\sqrt{w^{T}\Omega w}$ being positive to note that $z\leq\sqrt{w^{T}\Omega w}$ can be squared on both sides:
$$
    \min_w \frac{1}{2}w^{T}\Sigma w - \lambda w^{T}\mu - \lambda\kappa z\ \text{ s.t. }\ z^2\leq w^{T}\Omega w,\ Mw = \begin{bmatrix} 0.5 \\ 0.5\end{bmatrix},\ l\leq w\leq u.
$$

To explore this with our toy problem from before, we will define
$$
    \bar{\mu} = \begin{bmatrix} 1 \\ 5 \\ 2 \end{bmatrix},\quad
    \Omega = \begin{bmatrix} 1 & 0 & 0 \\ 0 & 4 & 0 \\ 0 & 0 & \frac{1}{8} \end{bmatrix},\quad
    \kappa = 0.5
$$
and retain the other variables as before.

In [7]:
omega = np.array([
    [1, 0, 0],
    [0, 4, 0],
    [0, 0, 1/8]
])

kappa = 0.5

We then formulate this in Python as follows.

In [8]:
# create a new model for robust genetic selection
model = gp.Model("robustGS")

# define variables of interest as a continuous
w = model.addMVar(shape=problem_size, lb=0.0, vtype=GRB.CONTINUOUS, name="w")
z = model.addVar(lb=0.0, name="z")

# setup the robust objective function
model.setObjective(
    0.5*w@(relationship_matrix@w) - lam*w.transpose()@expected_breeding_values - lam*kappa*z,
GRB.MINIMIZE)

# add quadratic uncertainty constraint
model.addConstr(z**2 <= w.transpose()@omega@w, name="uncertainty")
# add sub-to-half constraints
model.addConstr(M @ w == m, name="sum-to-half")
# add weight-bound constraints
model.addConstr(w >= lower_bound, name="lower bound")
model.addConstr(w <= upper_bound, name="upper bound")

# solve the problem with Gurobi
model.optimize()
print(f"w = {w.X},\nz = {z.X}.")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i5-8350U CPU @ 1.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 22 rows, 4 columns and 24 nonzeros
Model fingerprint: 0xf7819a35
Model has 3 quadratic objective terms
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e-01, 4e+00]
  Objective range  [2e-01, 2e+00]
  QObjective range [1e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 1e+00]
Presolve removed 21 rows and 1 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 21 rows and 1 columns
Presolve time: 0.00s
Presolved: 4 rows, 6 columns, 9 nonzeros
Presolved model has 2 quadratic objective terms
Presolved model has 1 quadratic constraint(s)
Presolved model has 2 bilinear constraint(s)
Variable types: 6 continuous, 0 integ

We can repeat this experiment with varying $\kappa$ to see how how different tolerances for uncertainty impact the robust turning point returned.

| $\kappa$ |        $w$            |   $z$   | $f(w,z)$ | Gap |
| ---: | ------------------------: | ------: | -------: | --: |
|  0.0 | [0.50000 0.37500 0.12500] | 0.0 | -8.12500000e-01 | 0% |
|  0.5 | [0.50000 0.42869 0.07131] | 0.9928459846964108 | -1.049180143116e+00 | 0.0041% |
|  1.0 | [0.50000 0.48606 0.01394] | 1.0931753188733573 | -1.309752491846e+00 | 0.0023% |
|  2.0 | [0.50000 0.50000 0.00000] | 1.1180339796124110 | -1.868033983797e+00 | 0.0005% |
|  4.0 | [0.50000 0.50000 0.00000] | 1.1180339856336740 | -2.986067972548e+00 | 0.0021%

We can see that in the case $\kappa = 0$ we return the standard optimization solution, as expected. We also note that there are not further changes once $\kappa$ falls above a certain threshold, indicating that we have successfully converged to the "riskiest" portfolio.

## Real (Simulated) Data

Now we've looked at how these problems can be approached with Gurobi, we can try an example with realistic simulated data. Here we have data for a cohort of 50 candidates split across three data files, each containing space-separated values.

1. In `A50.txt` the matrix $\Sigma$ is described, with columns for $i$, $j$, and $a_{ij}$, where only the upper triangle is described since the matrix is symmetric.
2. In `EBV50.txt` the vector $\bar{\mu}$ is described, with only one column containing the posterior mean over 1000 samples.
3. In `S50.txt` the matrix $\Omega$ is described, with columns for $i$, $j$, and $\sigma_{ij}$, where only the upper triangle is described since the matrix is symmetric.

Note that this particular problem does not contain a separate file for sex data; odd indexed candidates are sires and even indexed candidates are dams. For now, we also don't have a $l$ or $u$ bounding the possible weights.


The first step then is clearly going to be loading the matrices in particular from file. We create the following function to do so.


In [9]:
def load_problem(A_filename, E_filename, S_filename, dimension=False):
    """
    Used to load genetic selection problems into NumPy. It takes three
    string inputs for filenames where Sigma, Mu, and Omega are stored,
    as well as an optional integer input for problem dimension if this
    is known. If it's know know, it's worked out based on E_filename.

    As output, it returns (A, E, S, n), where A and S are n-by-n NumPy
    arrays, E is a length n NumPy array, and n is an integer.
    """

    def load_symmetric_matrix(filename, dimension):
        """
        Since NumPy doesn't have a stock way to load matrices
        stored in coordinate format format, this adds one.
        """

        matrix = np.zeros([dimension, dimension])

        with open(filename, 'r') as file:
            for line in file:
                i, j, entry = line.split(" ")
                # data files indexed from 1, not 0
                matrix[int(i)-1, int(j)-1] = entry
                matrix[int(j)-1, int(i)-1] = entry

        return matrix


    # if dimension wasn't supplied, need to find that
    if not dimension:
        # get dimension from EBV, since it's the smallest file
        with open(E_filename, 'r') as file:
            dimension = sum(1 for _ in file)

    # EBV isn't in coordinate format so can be loaded directly
    E = np.loadtxt(E_filename)  
    # A and S are stored by coordinates so need special loader
    A = load_symmetric_matrix(A_filename, dimension)
    S = load_symmetric_matrix(S_filename, dimension)

    return A, E, S, dimension

For the given example problem we can now solve it with Gurobi using the exact same methods as before, for both the standard genetic selection problem and the robust version of the problem. The following cell does both alongside each other to accentuate the differences.

In [10]:
sigma, mubar, omega, n = load_problem(
    "../Example/A50.txt",
    "../Example/EBV50.txt",
    "../Example/S50.txt",
    50)

lam = 0.5
kappa = 2

# define the M so that column i is [1;0] if i is a sire (so even) and [0;1] otherwise 
M = np.zeros((2, n))
M[0, range(0,50,2)] = 1
M[1, range(1,50,2)] = 1
# define the right hand side of the constraint Mx = m
m = np.array([[0.5], [0.5]])

# create models for standard and robust genetic selection
model_std = gp.Model("n50standard")
model_rbs = gp.Model("n50robust")

# initialise w for both models, z for robust model
w_std = model_std.addMVar(shape=n, lb=0.0, vtype=GRB.CONTINUOUS, name="w") 
w_rbs = model_rbs.addMVar(shape=n, lb=0.0, vtype=GRB.CONTINUOUS, name="w")
z_rbs = model_rbs.addVar(lb=0.0, name="z")

# define the objective functions for both models
model_std.setObjective(
    0.5*w_std.transpose()@(sigma@w_std) - lam*w_std.transpose()@mubar,
GRB.MINIMIZE)

model_rbs.setObjective(
    # Gurobi does offer a way to set one objective in terms of another, i.e.
    # we could use `model_std.getObjective() - lam*kappa*z_rbs` to define this
    # robust objective, but it results in a significant slowdown in code.
    0.5*w_rbs.transpose()@(sigma@w_rbs) - lam*w_rbs.transpose()@mubar - lam*kappa*z_rbs,
GRB.MINIMIZE)

# add sum-to-half constraints to both models
model_std.addConstr(M @ w_std == m, name="sum-to-half")
model_rbs.addConstr(M @ w_rbs == m, name="sum-to-half")

# add quadratic uncertainty constraint to the robust model
model_rbs.addConstr(z_rbs**2 <= w_rbs.transpose()@omega@w_rbs, name="uncertainty")

# since working with non-trivial size, set a time limit
time_limit = 60*5  # 5 minutes
model_std.setParam(GRB.Param.TimeLimit, time_limit)
model_std.setParam(GRB.Param.TimeLimit, time_limit)

# for the same reason, also set a duality gap tolerance
duality_gap = 0.05
model_std.setParam('MIPGap', duality_gap)
model_rbs.setParam('MIPGap', duality_gap)

# solve both problems with Gurobi
model_std.optimize()
model_rbs.optimize()

# HACK code which prints the results for comparison in a nice format
print("\nSIRE WEIGHTS\t\t\t DAM WEIGHTS")
print("-"*20 + "\t\t " + "-"*20)
print(" i   w_std    w_rbs\t\t  i   w_std    w_rbs")
for candidate in range(25):
    print(f"{candidate*2:02d}  {w_std.X[candidate*2]:.5f}  {w_rbs.X[candidate*2]:.5f} \
            {candidate*2+1:02d}  {w_std.X[candidate*2+1]:.5f}  {w_rbs.X[candidate*2+1]:.5f}")
    
print(f"\nMaximum change: {max(np.abs(w_std.X-w_rbs.X)):.5f}")
print(f"Average change: {np.mean(np.abs(w_std.X-w_rbs.X)):.5f}")
print(f"Minimum change: {min(np.abs(w_std.X-w_rbs.X)):.5f}")

Set parameter TimeLimit to value 300
Set parameter MIPGap to value 0.05
Set parameter MIPGap to value 0.05
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i5-8350U CPU @ 1.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 50 columns and 100 nonzeros
Model fingerprint: 0x534ff85e
Model has 1275 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-01, 1e+00]
  QObjective range [5e-02, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 5e-01]
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 50 columns, 50 nonzeros
Presolved model has 1275 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 49
 AA' NZ     : 1.274e+03
 Factor NZ  : 1.326e+03
 Factor Ops : 4.553e+04 (less than 1 second per iteration)
 Threa

We can see that with parameters $\lambda = 0.5, \kappa = 2$, there's a more significant shift in the portfolios produced and we see in the return maximization version of the problem. It also takes Gurobi significantly longer to compute. More testing would be needed to confirm why this was the case.